In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from copy import deepcopy
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_log_error, mean_squared_error

In [2]:
seed=163

In [3]:
# Modelling ALgorithm

parametrs = {
    'learning_rate': [0.05, 0.1, 0.2],
    'subsample': [1, 0.75, 0.5]
}

parametrs_v2 = {
    'estimator__learning_rate': [0.05, 0.1, 0.2],
    'estimator__subsample': [1, 0.75, 0.5]
}
FUNCTION = GridSearchCV(XGBRegressor(base_score=.95, 
                                     n_jobs=4, 
                                     n_random_state=seed, 
                                     n_estimators=500, 
                                     subsample=0.8, 
                                     reg_alpha=0.001, 
                                     reg_lambda=0.001), 
                        parametrs)

FUNCTION_v2 = GridSearchCV(MultiOutputRegressor(XGBRegressor(base_score=.95, 
                                                             n_jobs=4, 
                                                             n_random_state=seed, 
                                                             n_estimators=500, 
                                                             subsample=0.8, 
                                                             reg_alpha=0.001, 
                                                             reg_lambda=0.001)), 
                           parametrs_v2)

# Data

In [4]:
train_df = pd.read_csv("../../Data/Fixed-Fixed.csv")
train_df.head()

,depth,l1,l2,Mode,Frequency,Max_Deflection,Average_Deflection
0,0.0,0,0,1,52.976,18.084,9.0351
1,0.0,0,0,2,145.920,17.293,9.3771
2,0.0,0,0,3,285.990,17.430,9.5803
3,0.0,0,0,4,323.400,28.366,10.8720
4,0.0,0,0,5,472.810,17.537,9.7190


In [5]:
test_df = pd.read_csv("../../Data/Fixed-Fixed-Test.csv")
test_df.head()

,depth,l1,l2,Mode,Frequency,Max_Deflection,Average_Deflection
0,0.5,63,510,1,52.613,7.5540,3.8030
1,0.5,63,510,2,144.790,7.2355,3.6191
2,0.5,63,510,3,283.540,7.3149,4.0421
3,0.5,63,510,4,322.530,11.8550,4.5453
4,0.5,63,510,5,468.260,7.4041,3.8655


In [6]:
train_obj = train_df.groupby("Mode")
test_obj = test_df.groupby("Mode")

# Utility

In [7]:
class Experiment:
    def __init__(self, data, x_args, y_args, split, seed):
        x = data[x_args].values
        y = data[y_args].values
        _X_train, _X_test, _y_train, self.Y_test = train_test_split(x, y, 
                                                                     test_size=split, 
                                                                     random_state=seed)
        
        self.x_args = x_args
        self.y_args = y_args
        
        self.std_X = StandardScaler()
        self.X_train = self.std_X.fit_transform(_X_train)
        self.X_test = self.std_X.transform(_X_test)
        
        self.std_Y = StandardScaler()
        self.Y_train = self.std_Y.fit_transform(_y_train)
        
        self.model = None
        
        
    def print_result(self, pred, true):
        for i in range(true.shape[-1]):
            print("- Explained Variance Score: {:0.6f}%".format(100*explained_variance_score(true[:,i], pred[:,i])))
            print("- Mean Absolute Error: {:0.6f}".format(mean_absolute_error(true[:,i], pred[:,i])))
            print("- Mean Squared Error: {:0.6f}".format(mean_squared_error(true[:,i], pred[:,i])))
            print("")
        
    def train_model(self, algorithm):
        self.model = algorithm
        self.model.fit(self.X_train, self.Y_train)
        
    def validation(self):
        res = self.model.predict(self.X_test)
        preds = self.std_Y.inverse_transform(res)
        preds = preds.reshape(self.Y_test.shape)
        
        print("\nVALIDATION RESULT")
        self.print_result(preds, self.Y_test)
        
    def testing(self, data):
        x = data[self.x_args].values
        y = data[self.y_args].values
        
        x = self.std_X.transform(x)
        
        res = self.model.predict(x)
        preds = self.std_Y.inverse_transform(res)
        preds = preds.reshape(y.shape)
        
        print("\nTEST RESULT")
        self.print_result(preds, y)

# Multi Modes Model

In [8]:
temp = Experiment(train_df, 
                  x_args=['depth', 'l1', 'l2', 'Mode'], 
                  y_args=['Frequency'], 
                  split=0.10, 
                  seed=seed)

temp.train_model(FUNCTION)
temp.validation()
temp.testing(test_df)


VALIDATION RESULT
- Explained Variance Score: 99.997004%
- Mean Absolute Error: 0.573368
- Mean Squared Error: 0.787848


TEST RESULT
- Explained Variance Score: 99.983327%
- Mean Absolute Error: 0.906741
- Mean Squared Error: 4.260458



# Single Mode Models

In [9]:
# Data Transformation

def transform(dataframe):
    dataframe_obj = dataframe.groupby("Mode")
    temp = deepcopy(dataframe_obj.get_group(1)[["depth", "l1", "l2", "Frequency"]])
    temp.pop("Frequency")
    temp = temp.reset_index()
    temp.pop("index")


    for i in range(1, 7):
        temp["Frequency_{}".format(i)] = 0
        temp["Frequency_{}".format(i)] = dataframe_obj.get_group(i)["Frequency"].reset_index()["Frequency"]
        
    return temp

In [10]:
train_df = transform(train_df)
test_df = transform(test_df)

In [11]:
train_df.head()

,depth,l1,l2,Frequency_1,Frequency_2,Frequency_3,Frequency_4,Frequency_5,Frequency_6
0,0.0,0,0,52.976,145.92,285.99,323.40,472.81,489.85
1,0.5,60,110,52.618,144.89,283.94,322.78,469.30,489.23
2,0.5,40,260,52.709,145.17,284.50,322.86,470.23,489.26
3,0.5,80,440,52.689,144.99,284.09,322.73,469.01,489.25
4,0.5,40,520,52.653,144.92,283.70,322.75,468.69,489.24


In [12]:
temp = Experiment(train_df, 
                  x_args=['depth', 'l1', 'l2'], 
                  y_args=['Frequency_1', 'Frequency_2', 'Frequency_3','Frequency_4', 'Frequency_5', 'Frequency_6'], 
                  split=0.10, 
                  seed=seed)

temp.train_model(FUNCTION_v2)
temp.validation()
temp.testing(test_df)


VALIDATION RESULT
- Explained Variance Score: 96.698984%
- Mean Absolute Error: 0.053411
- Mean Squared Error: 0.004559

- Explained Variance Score: 95.029565%
- Mean Absolute Error: 0.162269
- Mean Squared Error: 0.036902

- Explained Variance Score: 92.725610%
- Mean Absolute Error: 0.398664
- Mean Squared Error: 0.261943

- Explained Variance Score: 97.222312%
- Mean Absolute Error: 0.090905
- Mean Squared Error: 0.018115

- Explained Variance Score: 89.047104%
- Mean Absolute Error: 0.792963
- Mean Squared Error: 0.871603

- Explained Variance Score: 97.484717%
- Mean Absolute Error: 0.160652
- Mean Squared Error: 0.048580


TEST RESULT
- Explained Variance Score: 79.752797%
- Mean Absolute Error: 0.095267
- Mean Squared Error: 0.024064

- Explained Variance Score: 66.154606%
- Mean Absolute Error: 0.411449
- Mean Squared Error: 0.475904

- Explained Variance Score: 48.076453%
- Mean Absolute Error: 0.973660
- Mean Squared Error: 2.947697

- Explained Variance Score: 18.940932%
- 